In [ ]:
!pip install optuna pytorch-lightning pytorch-forecasting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.3 MB/s eta 0:00:00


In [37]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
from torch.utils.data import DataLoader, TensorDataset
import optuna

Загрузка данных и создание датасета

In [38]:
df = pd.read_csv('daily_accidents.csv', parse_dates=['CRASH DATE'])
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df.set_index('CRASH DATE', inplace=True)

df = df[['ACCIDENT_COUNT']]

In [39]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaled = scaler.fit_transform(df)

In [40]:
X = []
y = []

seq_length = 30

for i in range(len(df_scaled) - seq_length):
    X.append(df_scaled[i:i + seq_length])
    y.append(df_scaled[i + seq_length])

In [41]:
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512)

Модель

In [42]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))

        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)

        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class LSTransformerFusion(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, output_size, dropout=0.1):
        super().__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)

        self.positional_encoding = PositionalEncoding(d_model=hidden_size, dropout=dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=hidden_size * 2,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

        self.fusion_fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        transformer_input = self.positional_encoding(x)
        transformer_out = self.transformer_encoder(transformer_input)

        lstm_last = lstm_out[:, -1, :]
        transformer_last = transformer_out[:, -1, :]
        fused = torch.cat([lstm_last, transformer_last], dim=-1)

        return self.fusion_fc(fused)


Функция подбора гиперпараметров

In [ ]:
def objective_lstransformer(trial):
    num_heads = trial.suggest_categorical("num_heads", [2, 4, 8])

    hidden_size_options = [hs for hs in range(32, 257, 8) if hs % num_heads == 0]
    hidden_size = trial.suggest_categorical("hidden_size", hidden_size_options)

    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = LSTransformerFusion(
        input_size=X_train.shape[2],
        hidden_size=hidden_size,
        num_layers=num_layers,
        num_heads=num_heads,
        output_size=1
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    model.train()
    for epoch in range(20):
        for inputs, labels in train_loader:
            inputs, labels = inputs, labels

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

    return test_loss / len(test_loader)


Подбор гиперпараметров

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_lstransformer, n_trials=30)

print("Best parameters:", study.best_params)

[I 2025-05-13 18:31:31,634] A new study created in memory with name: no-name-c46b6be4-e73e-436c-a334-0da56166f6bc
[I 2025-05-13 18:34:35,203] Trial 0 finished with value: 0.003140741609968245 and parameters: {'num_heads': 4, 'hidden_size': 128, 'num_layers': 2, 'lr': 0.0003433234531632159}. Best is trial 0 with value: 0.003140741609968245.
[I 2025-05-13 18:35:57,013] Trial 1 finished with value: 0.0030078409472480416 and parameters: {'num_heads': 8, 'hidden_size': 56, 'num_layers': 3, 'lr': 0.0021474865587272404}. Best is trial 1 with value: 0.0030078409472480416.
[I 2025-05-13 18:36:33,983] Trial 2 finished with value: 0.0031521099153906107 and parameters: {'num_heads': 2, 'hidden_size': 32, 'num_layers': 3, 'lr': 0.0018211195136755044}. Best is trial 1 with value: 0.0030078409472480416.
[I 2025-05-13 18:42:59,505] Trial 3 finished with value: 0.0031176169868558645 and parameters: {'num_heads': 2, 'hidden_size': 224, 'num_layers': 2, 'lr': 0.0016106530500180599}. Best is trial 1 with 

Best parameters: {'num_heads': 8, 'hidden_size': 56, 'num_layers': 3, 'lr': 0.0021474865587272404}


Параметры модели и инициализация

In [43]:
input_size = 1
hidden_size = 56
num_layers = 3
num_heads = 8
output_size = 1
learning_rate = 0.0021474865587272404
batch_size = 256

In [44]:
model = LSTransformerFusion(input_size, hidden_size, num_layers, num_heads, output_size)

In [45]:
criterion = nn.MSELoss()

In [46]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Обучение

In [58]:
epochs = 150

In [48]:
start_time = time.time()
train_loss = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    batch_count = 0

    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]


        optimizer.zero_grad()

        output = model(X_batch)

        if y_batch.ndim == 1:
            y_batch = y_batch.unsqueeze(1)

        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        batch_count += 1

    avg_loss = running_loss / batch_count
    train_loss.append(avg_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

Epoch 10/150, Loss: 0.0287
Epoch 20/150, Loss: 0.0276
Epoch 30/150, Loss: 0.0208
Epoch 40/150, Loss: 0.0252
Epoch 50/150, Loss: 0.0190
Epoch 60/150, Loss: 0.0199
Epoch 70/150, Loss: 0.0201
Epoch 80/150, Loss: 0.0198
Epoch 90/150, Loss: 0.0206
Epoch 100/150, Loss: 0.0200
Epoch 110/150, Loss: 0.0194
Epoch 120/150, Loss: 0.0201
Epoch 130/150, Loss: 0.0194
Epoch 140/150, Loss: 0.0199
Epoch 150/150, Loss: 0.0192
Training completed in 550.93 seconds


Предсказание и метрики

In [49]:
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train).detach().numpy()
    y_pred_test = model(X_test).detach().numpy()

In [50]:
y_pred_train = scaler.inverse_transform(y_pred_train)
y_pred_test = scaler.inverse_transform(y_pred_test)
y_train = scaler.inverse_transform(y_train.numpy())
y_test = scaler.inverse_transform(y_test.numpy())

In [53]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

correlation_train = np.corrcoef(y_train.flatten(), y_pred_train.flatten())[0, 1]
correlation_test = np.corrcoef(y_test.flatten(), y_pred_test.flatten())[0, 1]

print(f'Train RMSE: {rmse_train:.4f}, MAE: {mae_train:.4f}, R²: {r2_train:.4f}, Correlation: {correlation_train:.4f}')
print(f'Test RMSE: {rmse_test:.4f}, MAE: {mae_test:.4f}, R²: {r2_test:.4f}, Correlation: {correlation_test:.4f}')

Train RMSE: 69.9796, MAE: 49.9274, R²: 0.8118, Correlation: 0.9079
Test RMSE: 30.1629, MAE: 23.5648, R²: 0.2705, Correlation: 0.5634
